**Sprint 3 / Tasca 2: Dataframes i anàlisi estadístic**

*Tasca 2*

**Exercici 1**

Descarrega el data set Airlines Delay: Airline on-time statistics and delay causes i carrega’l a un Pandas Dataframe. Explora les dades que conté, explica breument quines variables hi ha i queda’t únicament amb les columnes que consideris rellevants. Justifica la teva elecció.

In [ ]:
from pathlib import Path
import numpy as np
import pandas as pd
from scipy import stats
import random
import datetime as dt
import math


In [ ]:
# file paths
data_path = 'D:/Sistema_Solar/Python/itacademy/sprint03/data/'
data_path = Path(data_path)

In [ ]:
file_name = 'DelayedFlights.csv'
file = data_path / file_name
delay_df = pd.read_csv(file)

In [ ]:
pd.set_option('display.max_columns', None)
pd.set_option('display.expand_frame_repr', False)
delay_df.describe(include='all')

In [ ]:
delay_df

Redueix la dimensió del dataset de manera aleatòria per tal d'obtenir un dataset de només 200.000 registres. Tots els exercicis s'han de fer amb aquest dataset reduït.

In [ ]:
delay_df = delay_df.sample(n=200000)

In [ ]:
delay_df

El dataset està conformat per 29 variables:

0. Unnamed 0: Camp que ennumera els registres
1. Year: L'any
2. Month: El mes
3. DayofMonth: Dia del mes
4. DayofWeek: Dia de la setmana
5. DepTime: L'hora de sortida real, en hora local (format hhmm)
6. CRSDepTime: L'hora de sortida programda
7. ArrTime: L'hora d'arribada real, en hora local
8. CRSArrTime: L'hora d'arribada programada
9. UniqueCarrier: Codi IATA de la companyia
10. FlightNum: Número del vol
11. TailNum: número de la cola del avió que identifica un avió.
12. ActualElapsedTime: Durada real del vol, en minuts
13. CRSElapsedTime: Durada del vol programat
14. AirTime: Temps en el qual l'avió es troba enlairat
15. ArrDelay: Retard en l'arribada, en minuts. Sols es considera un vol ha arribat amb retard si ho fa en 15 o més minuts del programat.
16. DepDelay: Retard en la sortida
17. Origin: Codi IATA de l'aeroport de sortida
18. Dest: Codi IATA de l'aeroport d'arribada
19. Distance: Distància del vol, en milles
20. TaxiIn: Temps en el qual l'avió circula per pista en direcció a la pista d'enlairament.
21. TaxiOut: Temps en el qual l'avió circula per pista des de l'aterratge fins el lloc de desembarcament.
22. Cancelled: 1 si el vol ha sigut cancel·lat o 0 si no ho ha estat.
23. CancellationCode: Codi de cancel·lació indicant la raó (A = Aerolinia; B = Temps; C = Retards en el Sistema d'espai aeri nacional que inclou condicions meteorologiques no extremes, operacions en l'aeroport, alt volum de tr'afic, etc...; D = Seguretat)
24. Diverted: Indica si el vol ha sigut desviat (1 = Sí, 0 = No).
25. CarrierDelay: El temps de retard degut a motius de l'aerolínia.
26. WeatherDelay: El temps de retard degut a motius del temps.
27. NASDelay: El temps de retard degut al NAS, el Sistema d'espai aeri nacional.
28. SecurityDelay: El temps de retard degut a motius de seguretat.
29. LateAircraftDelay: El temps de retard en un aeroport degut al retard en l'aeroport anterior d'on ve un avió. 

**Selecció de variables i creació de noves**



La columna 0 s'el·limina. Les columnes 1 a 3 ens indiquen el dia i l'hora del vol, les mantenim però el·liminem (4) el dia de la setmana.

Mentre que les de la 5 a la 8, ens quedem simplement amb l'hora esperada ja  que ja obtenim els retards en un altre camp. Però mantenim l'hora d'arribava real per fer comprovacions posteriors.



In [ ]:
delay_df = delay_df.drop(columns=['Unnamed: 0', 'DayOfWeek', 'DepTime', 'CRSDepTime']) 

### BORRAR Ací mire el número de vol i veig que no vol dir el que creia. Així que el llevaré

Podem calcular el número de vols diferents que hi han a partir del número de vol

In [ ]:
print(len(delay_df['FlightNum'].unique()))


Com també quin són els vols més freqüents:

In [ ]:
flight_num_count = delay_df['FlightNum'].value_counts()
print(flight_num_count)

Quin són aquests trajectes? Necessitem les columnes Origin i Dest que són les mateixes per a cada número de vol. Ho comprovem amb el número de vol mé nombròs.

In [ ]:
search = delay_df['FlightNum'] == flight_num_count.iloc[0]
busiest_flight_df = delay_df[search]

In [ ]:
busiest_flight_df['Origin']

### Continuem

Mantenim (9) UniqueCarrier per identificar l'aerolinia i (11) el número de cola. Descartem el (10) número de vol. Del (12) i (13), la diferència és el retard però mantenim (14) per calcular posteriorment la velocitat mitjana. Retindrem (15) ArrDelay i (16) DepDelay.

In [ ]:
delay_df = delay_df.drop(columns=['CRSElapsedTime', 'FlightNum', 'ActualElapsedTime'])

Mantenim els codis IATA dels aeroports d'origen i de sortida (17 i 18). Mantenim la distància per calcular la velocitat (19) però descartem el temps que està l'avió circulant a l'aeroport (20 i 21)

In [ ]:
delay_df = delay_df.drop(columns=['TaxiIn', 'TaxiOut'])

Retenim (22) i (23) per saber quins vol són cancel·lats, i els motius. i també els retards atribuïts als diferents mtoius (25-29).

In [ ]:
delay_df = delay_df.drop(columns=['Diverted'])

**Exercici 2**

Fes un informe complet del dataset:

- Resumeix estadísticament el dataset i les columnes d’interès. 
- Fes una anàlisi estadístic del que consideris rellevant.
- Troba quantes dades faltants hi ha per columna.
- Crea columnes noves (velocitat mitjana del vol, si ha arribat tard o no...).
- Fes una taula de les aerolínies amb més endarreriments acumulats.
- Quins són els vols més llargs? I els més endarrerits? Busca les rutes més llargues i les que acumulen més retards.
- Aporta allò que consideris rellevant.

Creem una nova columna 'Date' que uneix la informació de les columnes 1 a 4 en una sola. Aquestes columnes les el·liminem.

In [ ]:
delay_df['Date'] = pd.to_datetime(pd.DataFrame({'year':delay_df['Year'], 'month':delay_df['Month'], 'day':delay_df['DayofMonth']}))

delay_df = delay_df.drop(columns=['Year', 'Month', 'DayofMonth']) 

# També reordenem les columnes

In [ ]:
cols = delay_df.columns.tolist()
cols = cols[-1:] + cols[:-1]
delay_df = delay_df[cols]

In [ ]:
delay_df

Quina és la completitud de les dades disponibles? Cerquem els NA de les diferents variables?

In [ ]:
delay_df.isna().sum()

delay_df.isna().sum() / len(delay_df) * 100

1. Date                  
2. CRSArrTime            
3. UniqueCarrier         
4. FlightNum             
5. TailNum               
6. ArrDelay              
7. DepDelay             
8. Origin               
9. Dest                  
10. Cancelled             
11. CancellationCode      
12. CarrierDelay         
13. WeatherDelay         
14. NASDelay             
15. SecurityDelay        
16. LateAircraftDelay   

Però açò pot estar relacionat a que falten dades o bé perquèe, en els casos on apareixen NA, relacionat amb que no s'ha registrat retards. Anem a inspeccionar les diferents variables.

La data de les dades és de l'any 2008

In [ ]:
delay_df['Date'].min()

In [ ]:
delay_df['Date'].max()

2. CRSArrTime: El temps d'arribada esperada

In [ ]:
delay_df['CRSArrTime'] = delay_df['CRSArrTime']/100

In [ ]:
delay_df['CRSArrTime']

In [ ]:
# alternativa per ficar-ho en time format
m,s = map(int,"14.15".split("."))
m,s = (m+s)//60,s%60
dt.datetime.strptime('{}.{}'.format(m,s),'%H.%M').time()
# aplicat al meu dataset ??? No se si m'és necessari

3. UniqueCarrier. Les diferents companyies que conformen les dades són:

In [ ]:
np.sort(delay_df['UniqueCarrier'].unique())

Aquest és el codi IATA (**I**nternational **A**ir **T**ransport **A**ssociation). Perquè quede més clar de quines companyies parlem, anem a incloure-les a la nostra dataframe a partir de la informació recabada de les següents pàgines webs.
https://en.wikipedia.org/wiki/List_of_airline_codes

https://aspm.faa.gov/aspmhelp/index/ASQP___Carrier_Codes_And_Names.html

https://www.tvlon.com/resources/airlinecodes.htm

Confeccionem un excel i l'apugem.

In [ ]:
file_name = 'UniqueCarrier.csv'
file = data_path / file_name
unique_carrier_df = pd.read_csv(file, sep = ';')

In [ ]:
unique_carrier_df.iloc[0:2]

In [ ]:
air_carrier_names = []
for code in delay_df['UniqueCarrier']:
    filter = unique_carrier_df['IATA Code'] == code
    name = unique_carrier_df[filter]['Air Carrier Name']
    try: 
        air_carrier_names.append(name.values[0])
    except IndexError:
        air_carrier_names.append(np.nan)

In [ ]:
delay_df['air_carrier_names'] = air_carrier_names

4. NumTail

In [ ]:
print(delay_df['TailNum'].value_counts())
print(len(delay_df['TailNum'].unique()))

5. Quins són els trajectes més freqüents? Amb els camps d'origen i destí, crearem un nou, la ruta.

In [ ]:
delay_df['Route'] = delay_df['Origin'] + '-' + delay_df['Dest']

In [ ]:
delay_df['Route'].value_counts()

Estes sigles corresponen al codi de la IATA i les podem traduir als aeroports per fer més fàcil reconèixer què rutes són, utilitzem aquesta relació dels codis:
    
https://datahub.io/core/airport-codes#resource-airport-codes

In [ ]:
file_name = 'airport-codes_csv.csv'
file = data_path / file_name
IATA_airport_df = pd.read_csv(file, sep = ',')

In [ ]:
IATA_airport_df

In [ ]:
# airport_name = []
# for code in delay_df['Origin']:
#     search = IATA_airport_df['iata_code'] == code
#     name = IATA_airport_df[search]['name']
#     try: 
#         airport_name.append(name.values[0])
#     except IndexError:
#         airport_name.append(np.nan)
        
# delay_df['Origin_name'] = airport_name


# airport_name = []
# for code in delay_df['Dest']:
#     search = IATA_airport_df['iata_code'] == code
#     name = IATA_airport_df[search]['name']
#     try: 
#         airport_name.append(name.values[0])
#     except IndexError:
#         airport_name.append(np.nan)
        
# delay_df['Dest_name'] = airport_name

Tarda massa fer-ho per a tota la df, així que ho aplicarem directament als resultats

In [ ]:
busiest_routes = delay_df['Route'].value_counts()

In [ ]:
busiest_routes

In [ ]:
busiest_routes.index[0:5]

In [ ]:
len(delay_df[delay_df.Cancelled == 1])

In [ ]:
busiest_routes_names = []
length = 5

for i in busiest_routes.index[0:length]:
    print(i)
    route_name = []
    i = i.split('-')   
    search = IATA_airport_df['iata_code'] == i[0] 
    origin = IATA_airport_df[search]['name'].values[0]
    search = IATA_airport_df['iata_code'] == i[0] 
    dest = IATA_airport_df[search]['name'].values[0]
    busiest_routes_names.append(origin + ' - ' + dest)

In [ ]:
busiest_routes_names = []
length = 5

for i in range(length):
    code = busiest_routes.index[i]
    route_name = []
    code = code.split('-')   
    search = IATA_airport_df['iata_code'] == code[0] 
    origin = IATA_airport_df[search]['name'].values[0]
    search = IATA_airport_df['iata_code'] == code[1] 
    dest = IATA_airport_df[search]['name'].values[0]
    busiest_routes_names.append(origin + ' - ' + dest + ' (' + str(busiest_routes.values[i]) + ')')
    

In [ ]:
busiest_routes_names

Ara pense que també han d'existir les rutes inverses, i les podriem intentar sumar? Per exemple, la primera ruta és.
ORD-LGA. Per tant, ha d'exisir també la ruta LGA - ORD. Anem a buscar-lo.


In [ ]:
search = delay_df['Route'] == 'LGA-ORD'
delay_df['Route'][search]

In [ ]:
len(delay_df['Route'][search])

In [ ]:
route_preunique = delay_df['Route'].unique()
route_unique = []

for route in route_preunique:
    route_inv = route.split('-')
    route_inv = route_inv[1] + '-' + route_inv[0]
    if (route not in route_unique and route_inv not in route_unique):
        route_unique.append(route)

In [ ]:
len(route_preunique)

In [ ]:
len(route_unique)

In [ ]:
route_dict = {}
for route in route_unique:
    search = delay_df['Route'] == route
    count = len(delay_df['Route'][search])
    route = route.split('-')
    route_2 = route[1] + '-' + route[0]
    search = delay_df['Route'] == route_2
    count += len(delay_df['Route'][search])
    route_dict[route_2] = count

In [ ]:
len(delay_df[delay_df.Cancelled == 1])

In [ ]:
route_df = pd.DataFrame.from_dict(data=route_dict, orient='index', columns=['count'])

In [ ]:
busiest_routes_2 = route_df.sort_values(by=['count'], ascending = False)

In [ ]:
busiest_routes_names_2 = []
length = 10

for i in range(length):
    code = busiest_routes_2.index[i]
    route_name = []
    code = code.split('-')   
    search = IATA_airport_df['iata_code'] == code[0] 
    origin = IATA_airport_df[search]['name'].values[0]
    search = IATA_airport_df['iata_code'] == code[1] 
    dest = IATA_airport_df[search]['name'].values[0]
    busiest_routes_names_2.append(origin + ' - ' + dest + ' (' + str(busiest_routes_2.values[i]) + ')')

In [ ]:
busiest_routes_names_2

6. Estudiem els endarreriments...:

- Quin % de vols presenten endarreriments?
- Classificació dels retards
- Quines rutes tenen més endarreriments?
- Quines línies?
- Afecten a uns avions més que d'altres?


Creem una nova columna i classifiquem els endarreriments. Podem considerar que si arriba en menys de 15 minuts no hi ha hagut retard.

Menys de 15 minuts: Sense retard\
Entre 15 y 30 minuts: Retard lleuger\
Entre 30 y 60 minuts: Retard moderat\
Més de 60 minuts: Gran retard

Hi han dades NA, aquestes perquè ocorren? Poden ser vols que no hagen tingut endarreriment, s'hagin cancel·lat? Ens calen de nou les dades, reals d'arribada.

In [ ]:
delay_df[delay_df['ArrDelay'] > 15]

In [ ]:
nan_delay = delay_df[np.isnan(delay_df['ArrDelay'])]
nan_delay[np.isnan(nan_delay['ArrTime'])]

In [ ]:
len(delay_df[delay_df.Cancelled == 1])

Observem que la major part dels Nans es deuen a que falta l'hora d'arribada real, tot i que sí que hi han dades del retard en l'eixida. Així descartem els registres amb Nans.

In [ ]:
delay_df_notna = delay_df[delay_df['ArrDelay'].notna()]

In [ ]:
len(delay_df[delay_df.Cancelled == 1])

In [ ]:
delay_df.loc[delay_df.ArrDelay < 15, 'ArrDelay_Class'] = 'Sense retard'
delay_df.loc[(delay_df.ArrDelay >= 15) & (delay_df.ArrDelay < 30), 'ArrDelay_Class'] = 'Petit'
delay_df.loc[(delay_df.ArrDelay >= 30) & (delay_df.ArrDelay < 60), 'ArrDelay_Class'] = 'Mitjà'
delay_df.loc[delay_df.ArrDelay > 60, 'ArrDelay_Class'] = 'Gran'

In [ ]:
delay_class_count = delay_df['ArrDelay_Class'].value_counts()[['Sense retard', 'Petit', 'Mitjà', 'Gran']] 
delay_class_count

Un 35 % dels vols no tenen retards i més d'un 20 % tenen retards superiors a l'hora

In [ ]:
delay_class_count  / len(delay_df) * 100

- Retards classificats per aerolínia:

In [ ]:
carrier_delay_dict = {}

for carrier in delay_df['UniqueCarrier'].unique():
    subset = delay_df[delay_df['UniqueCarrier'] == carrier]
    delay_class_count = subset['ArrDelay_Class'].value_counts()[['Sense retard', 'Petit', 'Mitjà', 'Gran']] 
    carrier_delay_dict[carrier] = delay_class_count

In [ ]:
carrier_delay_df = pd.DataFrame.from_dict(data=carrier_delay_dict, orient='index')

In [ ]:
carrier_delay_df['Total'] = carrier_delay_df['Sense retard'] + carrier_delay_df['Petit'] + carrier_delay_df['Mitjà'] + carrier_delay_df['Gran']

In [ ]:
carrier_delay_df['Sense retard %'] = round(carrier_delay_df['Sense retard'] / carrier_delay_df['Total'] * 100, 1)
carrier_delay_df['Petit %'] = round(carrier_delay_df['Petit'] / carrier_delay_df['Total'] * 100, 1)
carrier_delay_df['Mitjà %'] = round(carrier_delay_df['Mitjà'] / carrier_delay_df['Total'] * 100, 1)
carrier_delay_df['Gran %'] = round(carrier_delay_df['Gran'] / carrier_delay_df['Total'] * 100, 1)

In [ ]:
carrier_delay_df.sort_values(by = 'Sense retard %', ascending = False)

In [ ]:
len(delay_df[delay_df.Cancelled == 1])

- Quines rutes tenen més retards?

In [ ]:
for route in route_unique:
    delay_df.loc[delay_df.Route  == route, 'RouteUnique'] = route
    route_split = route.split('-')
    route_2 = route_split[1] + '-' + route_split[0]
    delay_df.loc[delay_df.Route  == route_2, 'RouteUnique'] = route

In [ ]:
route_delay_dict = {}

for route in route_unique:
    subset = delay_df[delay_df['RouteUnique'] == route]
    delay_class_count = subset['ArrDelay_Class'].value_counts()
    route_delay_dict[route] = delay_class_count

In [ ]:
route_delay_df = pd.DataFrame.from_dict(data=route_delay_dict, orient='index')

In [ ]:
route_delay_df['Total'] = route_delay_df['Sense retard'] + route_delay_df['Petit'] + route_delay_df['Mitjà'] + route_delay_df['Gran']

In [ ]:
route_delay_df['Sense retard %'] = round(route_delay_df['Sense retard'] / route_delay_df['Total'] * 100, 1)
route_delay_df['Petit %'] = round(route_delay_df['Petit'] / route_delay_df['Total'] * 100, 1)
route_delay_df['Mitjà %'] = round(route_delay_df['Mitjà'] / route_delay_df['Total'] * 100, 1)
route_delay_df['Gran %'] = round(route_delay_df['Gran'] / route_delay_df['Total'] * 100, 1)

In [ ]:
route_delay_df.sort_values(by = 'Total', ascending = False)

Abans convé er algo amb els NaN, ja que afecta per a obtindre el valor Total

In [ ]:
route_delay_df = route_delay_df.fillna(0)

In [ ]:
route_delay_df['Total'] = route_delay_df['Sense retard'] + route_delay_df['Petit'] + route_delay_df['Mitjà'] + route_delay_df['Gran']

In [ ]:
route_delay_df['Sense retard %'] = round(route_delay_df['Sense retard'] / route_delay_df['Total'] * 100, 1)
route_delay_df['Petit %'] = round(route_delay_df['Petit'] / route_delay_df['Total'] * 100, 1)
route_delay_df['Mitjà %'] = round(route_delay_df['Mitjà'] / route_delay_df['Total'] * 100, 1)
route_delay_df['Gran %'] = round(route_delay_df['Gran'] / route_delay_df['Total'] * 100, 1)

In [ ]:
route_delay_df.sort_values(by = 'Total', ascending = False)

In [ ]:
route_delay_df.sort_values(by = 'Sense retard %', ascending = False)

I anem a excloure les rutes que tenen pocs viatge (Per davall del percentil 5).

In [ ]:
route_delay_df.Total.quantile(0.1)

route_delay_df_q01 = route_delay_df[route_delay_df.Total >= route_delay_df.Total.quantile(0.1)]

In [ ]:
route_delay_df_q01.sort_values(by = 'Sense retard %', ascending = False)

- Motius principals dels retards. Cercarem per cada vol, quin ha sigut el motiu més important del retard

In [ ]:
delay_df

In [ ]:
t = [False, False, False, False, True, True, False, True, False, False, False, False, False, False, False, False]
[i for i, x in enumerate(t) if x]
[4, 5, 7]

In [ ]:
columns_req = ['CarrierDelay', 'WeatherDelay', 'NASDelay', 'SecurityDelay', 'LateAircraftDelay']
col_req_index = []

for column in columns_req:
    lst = delay_df.columns == column
    col_req_index.append([i for i, x in enumerate(lst) if x][0])

In [ ]:
col_req_index 

In [ ]:
col_extract = []
for item in col_req_index:
    col_extract.append(flight[item])

In [ ]:
main_reason_delay = []
for flight in delay_df.iterrows():
    min_delays = []
    for item in col_req_index:
        min_delays.append(flight[1][item])
    print(min_delays)
    max_index = []
    max_value = min_delays[0]
    for i, val in ((i, val) for i, val in enumerate(min_delays) if val >= max_value):
        if val == max_value:
            max_index.append(i)
        else:
            max_val = val
            max_index = [i]
    print(max_index)
    print('---')
    main_reason_delay.append(max_index)

In [ ]:
main_reason_delay

In [ ]:


max_indices = []
if seq:
    max_val = seq[0]
    for i,val in ((i,val) for i,val in enumerate(seq) if val >= max_val):
        if val == max_val:
            max_indices.append(i)
        else:
            max_val = val
            max_indices = [i]
            
            

- Quina és la velocitat mitjana dels trajectes?

In [ ]:
delay_df['MeanSpeed'] = round((delay_df['Distance'] / delay_df['AirTime']) * (1.609344 * 60), 1)

In [ ]:
delay_df

- Quin és el percentatge i el motiu principal de cancel·lació? (A = Aerolinia; B = Temps; C = Retards en el Sistema d'espai aeri nacional que inclou condicions meteorologiques no extremes, operacions en l'aeroport, alt volum de tràfic, etc...; D = Seguretat)

In [ ]:
(len(delay_df[delay_df.Cancelled == 1]) / len(delay_df))*100

In [ ]:
cancod_delay_df = delay_df[delay_df.CancellationCode != 'N']['CancellationCode']

In [ ]:
round(cancod_delay_df.value_counts() / len(cancod_delay_df)*100, 1)

In [ ]:
delay_df['Cancelled'].sort_values(ascending = False)

In [ ]:
len(delay_df[delay_df.Cancelled == 1])

In [ ]:
len(delay_df[delay_df.Cancelled == 1]) / len(delay_df)